In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

In [2]:
pandas = pd.read_table("airline.dat")
database = db.Database("airline",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
exclude = (  ArrivalTimeHours_1   ==  -1  )
database.remove(exclude)
  
# Choice
chosenAlternative = ( (  BestAlternative_1   *  1  ) + (  BestAlternative_2   *  2  ) ) + (  BestAlternative_3   *  3  )

In [3]:
#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1= Beta('Constant1',0,None,None,1)
Constant2= Beta('Constant2',0,None,None,0)
Constant3= Beta('Constant3',0,None,None,0)
Fare= Beta('Fare',0,None,None,0)
Legroom = Beta('Legroom',0,None,None,0)
SchedDE = Beta('SchedDE',0,None,None,0)
SchedDL = Beta('SchedDL',0,None,None,0)
Total_TT1= Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT1_sq = Beta('Total_TT1_sq',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

DepartureTimeSensitive  = DefineVariable('DepartureTimeSensitive', q11_DepartureOrArrivalIsImportant   == \
                                         1 ,database)
ArrivalTimeSensitive  = DefineVariable('ArrivalTimeSensitive', q11_DepartureOrArrivalIsImportant   ==  2 ,database)
Missing  = DefineVariable('Missing',(  q11_DepartureOrArrivalIsImportant   !=  1  ) *\
                          (  q11_DepartureOrArrivalIsImportant   !=  2  ),database)
DesiredDepartureTime  = DefineVariable('DesiredDepartureTime',q12_IdealDepTime ,database)
DesiredArrivalTime  = DefineVariable('DesiredArrivalTime',q13_IdealArrTime ,database)
ScheduledDelay_1  = DefineVariable('ScheduledDelay_1',(  DepartureTimeSensitive   *\
                                                       (  DepartureTimeMins_1   -  DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   *\
                                    (  ArrivalTimeMins_1   -  DesiredArrivalTime   ) ),database)
ScheduledDelay_2  = DefineVariable('ScheduledDelay_2',(  DepartureTimeSensitive   *\
                                                       (  DepartureTimeMins_2   -  DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   *\
                                    (  ArrivalTimeMins_2   -  DesiredArrivalTime   ) ),database)
ScheduledDelay_3  = DefineVariable('ScheduledDelay_3',(  DepartureTimeSensitive   *\
                                                       (  DepartureTimeMins_3   -  DesiredDepartureTime   ) ) +\
                                   (  ArrivalTimeSensitive   *\
                                    (  ArrivalTimeMins_3   -  DesiredArrivalTime   ) ),database)
Opt1_SchedDelayEarly  = DefineVariable('Opt1_SchedDelayEarly',(  -(ScheduledDelay_1 )  *\
                                                               (  ScheduledDelay_1   <  0  ) ) /  60 ,database)
Opt2_SchedDelayEarly  = DefineVariable('Opt2_SchedDelayEarly',(  -(ScheduledDelay_2 )  *\
                                                               (  ScheduledDelay_2   <  0  ) ) /  60 ,database)
Opt3_SchedDelayEarly  = DefineVariable('Opt3_SchedDelayEarly',(  -(ScheduledDelay_3 )  *\
                                                               (  ScheduledDelay_3   <  0  ) ) /  60 ,database)
Opt1_SchedDelayLate  = DefineVariable('Opt1_SchedDelayLate',(  ScheduledDelay_1   *\
                                                             (  ScheduledDelay_1   >  0  ) ) /  60 ,database)
Opt2_SchedDelayLate  = DefineVariable('Opt2_SchedDelayLate',(  ScheduledDelay_2   *\
                                                             (  ScheduledDelay_2   >  0  ) ) /  60 ,database)
Opt3_SchedDelayLate  = DefineVariable('Opt3_SchedDelayLate',(  ScheduledDelay_3   *\
                                                             (  ScheduledDelay_3   >  0  ) ) /  60 ,database)

TripTimeHours_1_sq = DefineVariable('TripTimeHours_1_sq', TripTimeHours_1**2, database)

# Utilities
Opt1 = Constant1 + Fare * Fare_1 + Legroom * Legroom_1 + SchedDE * Opt1_SchedDelayEarly +\
SchedDL * Opt1_SchedDelayLate + Total_TT1 * TripTimeHours_1 + Total_TT1_sq*TripTimeHours_1_sq
Opt2 = Constant2 + Fare * Fare_2 + Legroom * Legroom_2 + SchedDE * Opt2_SchedDelayEarly +\
SchedDL * Opt2_SchedDelayLate + Total_TT2 * TripTimeHours_2
Opt3 = Constant3 + Fare * Fare_3 + Legroom * Legroom_3 + SchedDE * Opt3_SchedDelayEarly + SchedDL *\
Opt3_SchedDelayLate + Total_TT3 * TripTimeHours_3
V = {1: Opt1,2: Opt2,3: Opt3}
av = {1: 1,2: 1,3: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_airline_powerseries"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

               Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2      -2.21    0.299   -7.37 1.73e-13         0.298        -7.41   
Constant3      -2.43    0.308    -7.9 2.89e-15         0.312        -7.78   
Fare         -0.0193 0.000699   -27.6        0        0.0008        -24.1   
Legroom        0.227    0.025    9.09        0        0.0267         8.51   
SchedDE       -0.139   0.0157    -8.9        0        0.0165        -8.46   
SchedDL       -0.105   0.0128    -8.2 2.22e-16        0.0137        -7.63   
Total_TT1      -0.87    0.173   -5.03 4.91e-07         0.172        -5.05   
Total_TT1_sq  0.0744   0.0217    3.44 0.000589         0.022         3.38   
Total_TT2     -0.301     0.07    -4.3 1.69e-05        0.0701         -4.3   
Total_TT3     -0.302   0.0702    -4.3 1.69e-05        0.0701        -4.31   

              Rob. p-value  
Constant2         1.23e-13  
Constant3         7.11e-15  
Fare                     0  
Legroom                  0  
SchedDE

In [4]:
# compare with model_specific
-2*(-2320.447 - -2314.435) # > Chi^2(0.95,1)=3.841 => Reject H0 that travel time is linear

12.024000000000342